In [3]:
import pandas
from neo4j import GraphDatabase
import time
import json
from ast import literal_eval

In [4]:
#do we have polygons for neighborhoods?
#look deeper into wiki data

In [6]:
df = pandas.read_csv('/Users/jamesades/Downloads/osm_sd.csv')
df = df[~df['names'].str.contains('Bathysphere training center', na = False)]
df = df[~df['names'].str.contains('Pacific Portal', na = False)]
df = df[~df['names'].str.contains('El Mazateño', na = False)]
df = df[~df['names'].str.contains('Nieves de garrafa', na = False)]
df = df[~df['names'].str.contains('Estacionamiento', na = False)]
df = df[~df['names'].str.contains(r'''\\''', na = False)]
dffilter = df['names'].str.contains('local')
df = df[dffilter]

df

,geometry_id,class,subclass,names,metadata,quadkey,original_source_tags,wkt
0,n4840618776@4,local,suburb,"{""local"":""La Jolla""}","{""quadkey"":""023013221201101"",""wikidata"":""Q8406...",2.301322e+13,"{""place"":""suburb"",""wikidata"":""Q840668"",""wikipe...",POINT (-117.2740777 32.8401623)
1,n8253475216@1,settlement,hamlet,"{""local"":""Rancho Verde""}","{""quadkey"":""023013221120001""}",2.301322e+13,"{""place"":""hamlet""}",POINT (-117.0559504 32.988474)
2,n150981480@4,settlement,hamlet,"{""local"":""Hillsdale""}","{""quadkey"":""023013221301320""}",2.301322e+13,"{""ele"":""170"",""gnis:County"":""San Diego"",""gnis:C...",POINT (-116.9280813 32.7792185)
3,n150965004@3,settlement,hamlet,"{""local"":""Agra""}","{""quadkey"":""023013202313020""}",2.301320e+13,"{""ele"":""51"",""gnis:Class"":""Populated Place"",""gn...",POINT (-117.5000627 33.3333724)
4,n8246746844@1,settlement,hamlet,"{""local"":""Sunrise Ranch""}","{""quadkey"":""023013221120120""}",2.301322e+13,"{""place"":""hamlet""}",POINT (-117.0217143 32.9681357)
...,...,...,...,...,...,...,...,...
742510,w974480734@1,commercial,furniture,"{""local"":""Jerome's Furniture""}","{""building"":""yes"",""landuse"":""developed:commerc...",2.301322e+13,"{""brand"":""Jerome's Furniture"",""brand:wikidata""...","POLYGON ((-117.0293862 32.637348, -117.0294134..."
742549,w622137659@1,commercial,convenience,"{""local"":""Glenns Market""}","{""building"":""yes"",""quadkey"":""023013221213030"",...",2.301322e+13,"{""building"":""yes"",""building_type"":""commercial""...","POLYGON ((-117.1320958 32.748159, -117.1320965..."
742604,w96330696@3,general,building,"{""local"":""X's & O's Lingerie and Gift Boutique""}","{""building"":""yes"",""quadkey"":""023013221011112"",...",2.301322e+13,"{""addr:city"":""Escondido"",""addr:country"":""US"",""...","POLYGON ((-117.0826846 33.1212324, -117.082854..."
742695,w97492542@5,amenity,fast_food,"{""local"":""Jack in the Box""}","{""amenity"":""fast_food"",""building"":""yes"",""quadk...",2.301322e+13,"{""addr:city"":""El Cajon"",""addr:country"":""US"",""a...","POLYGON ((-116.960768 32.8079634, -116.9608315..."


In [7]:
df.to_csv('graphs.csv', index = False)


df = pandas.read_csv('graphs.csv', converters={'names': literal_eval})

df

,geometry_id,class,subclass,names,metadata,quadkey,original_source_tags,wkt
0,n4840618776@4,local,suburb,{'local': 'La Jolla'},"{""quadkey"":""023013221201101"",""wikidata"":""Q8406...",2.301322e+13,"{""place"":""suburb"",""wikidata"":""Q840668"",""wikipe...",POINT (-117.2740777 32.8401623)
1,n8253475216@1,settlement,hamlet,{'local': 'Rancho Verde'},"{""quadkey"":""023013221120001""}",2.301322e+13,"{""place"":""hamlet""}",POINT (-117.0559504 32.988474)
2,n150981480@4,settlement,hamlet,{'local': 'Hillsdale'},"{""quadkey"":""023013221301320""}",2.301322e+13,"{""ele"":""170"",""gnis:County"":""San Diego"",""gnis:C...",POINT (-116.9280813 32.7792185)
3,n150965004@3,settlement,hamlet,{'local': 'Agra'},"{""quadkey"":""023013202313020""}",2.301320e+13,"{""ele"":""51"",""gnis:Class"":""Populated Place"",""gn...",POINT (-117.5000627 33.3333724)
4,n8246746844@1,settlement,hamlet,{'local': 'Sunrise Ranch'},"{""quadkey"":""023013221120120""}",2.301322e+13,"{""place"":""hamlet""}",POINT (-117.0217143 32.9681357)
...,...,...,...,...,...,...,...,...
127745,w974480734@1,commercial,furniture,{'local': 'Jerome's Furniture'},"{""building"":""yes"",""landuse"":""developed:commerc...",2.301322e+13,"{""brand"":""Jerome's Furniture"",""brand:wikidata""...","POLYGON ((-117.0293862 32.637348, -117.0294134..."
127746,w622137659@1,commercial,convenience,{'local': 'Glenns Market'},"{""building"":""yes"",""quadkey"":""023013221213030"",...",2.301322e+13,"{""building"":""yes"",""building_type"":""commercial""...","POLYGON ((-117.1320958 32.748159, -117.1320965..."
127747,w96330696@3,general,building,{'local': 'X's & O's Lingerie and Gift Boutique'},"{""building"":""yes"",""quadkey"":""023013221011112"",...",2.301322e+13,"{""addr:city"":""Escondido"",""addr:country"":""US"",""...","POLYGON ((-117.0826846 33.1212324, -117.082854..."
127748,w97492542@5,amenity,fast_food,{'local': 'Jack in the Box'},"{""amenity"":""fast_food"",""building"":""yes"",""quadk...",2.301322e+13,"{""addr:city"":""El Cajon"",""addr:country"":""US"",""a...","POLYGON ((-116.960768 32.8079634, -116.9608315..."


In [8]:
#convert some of the JSON strings stored in our DataFrame into Python dicts
tempdf = df.copy()

def tryconvert(value, default):
    try:
        return json.loads(value)
    except Exception as e:
        pass
    return default

#tempdf['names'] = tempdf['names'].apply(lambda x: tryconvert(x, {'name': "NA"}))

tempdf['original_tags_dict'] = tempdf['original_source_tags'].apply(lambda x: tryconvert(x, {}))

In [9]:
# And let's convert out Point geometry in WKT to a list of latitude/longitude coordinates
tempdf['point'] = tempdf['wkt'].apply(lambda x: x.replace('POINT (', "").replace(')', '').split())

In [10]:
tempdf.names.iloc[10]

{'local': 'Lindavista'}

In [11]:
tempdf

,geometry_id,class,subclass,names,metadata,quadkey,original_source_tags,wkt,original_tags_dict,point
0,n4840618776@4,local,suburb,{'local': 'La Jolla'},"{""quadkey"":""023013221201101"",""wikidata"":""Q8406...",2.301322e+13,"{""place"":""suburb"",""wikidata"":""Q840668"",""wikipe...",POINT (-117.2740777 32.8401623),"{'place': 'suburb', 'wikidata': 'Q840668', 'wi...","[-117.2740777, 32.8401623]"
1,n8253475216@1,settlement,hamlet,{'local': 'Rancho Verde'},"{""quadkey"":""023013221120001""}",2.301322e+13,"{""place"":""hamlet""}",POINT (-117.0559504 32.988474),{'place': 'hamlet'},"[-117.0559504, 32.988474]"
2,n150981480@4,settlement,hamlet,{'local': 'Hillsdale'},"{""quadkey"":""023013221301320""}",2.301322e+13,"{""ele"":""170"",""gnis:County"":""San Diego"",""gnis:C...",POINT (-116.9280813 32.7792185),"{'ele': '170', 'gnis:County': 'San Diego', 'gn...","[-116.9280813, 32.7792185]"
3,n150965004@3,settlement,hamlet,{'local': 'Agra'},"{""quadkey"":""023013202313020""}",2.301320e+13,"{""ele"":""51"",""gnis:Class"":""Populated Place"",""gn...",POINT (-117.5000627 33.3333724),"{'ele': '51', 'gnis:Class': 'Populated Place',...","[-117.5000627, 33.3333724]"
4,n8246746844@1,settlement,hamlet,{'local': 'Sunrise Ranch'},"{""quadkey"":""023013221120120""}",2.301322e+13,"{""place"":""hamlet""}",POINT (-117.0217143 32.9681357),{'place': 'hamlet'},"[-117.0217143, 32.9681357]"
...,...,...,...,...,...,...,...,...,...,...
127745,w974480734@1,commercial,furniture,{'local': 'Jerome's Furniture'},"{""building"":""yes"",""landuse"":""developed:commerc...",2.301322e+13,"{""brand"":""Jerome's Furniture"",""brand:wikidata""...","POLYGON ((-117.0293862 32.637348, -117.0294134...","{'brand': 'Jerome's Furniture', 'brand:wikidat...","[POLYGON, ((-117.0293862, 32.637348,, -117.029..."
127746,w622137659@1,commercial,convenience,{'local': 'Glenns Market'},"{""building"":""yes"",""quadkey"":""023013221213030"",...",2.301322e+13,"{""building"":""yes"",""building_type"":""commercial""...","POLYGON ((-117.1320958 32.748159, -117.1320965...","{'building': 'yes', 'building_type': 'commerci...","[POLYGON, ((-117.1320958, 32.748159,, -117.132..."
127747,w96330696@3,general,building,{'local': 'X's & O's Lingerie and Gift Boutique'},"{""building"":""yes"",""quadkey"":""023013221011112"",...",2.301322e+13,"{""addr:city"":""Escondido"",""addr:country"":""US"",""...","POLYGON ((-117.0826846 33.1212324, -117.082854...","{'addr:city': 'Escondido', 'addr:country': 'US...","[POLYGON, ((-117.0826846, 33.1212324,, -117.08..."
127748,w97492542@5,amenity,fast_food,{'local': 'Jack in the Box'},"{""amenity"":""fast_food"",""building"":""yes"",""quadk...",2.301322e+13,"{""addr:city"":""El Cajon"",""addr:country"":""US"",""a...","POLYGON ((-116.960768 32.8079634, -116.9608315...","{'addr:city': 'El Cajon', 'addr:country': 'US'...","[POLYGON, ((-116.960768, 32.8079634,, -116.960..."


In [12]:
class Neo4jConnection:
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exeception as e:
            print("Failed to create the driver:", e)
    
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
    
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            # TODO: convert to transaction function / 5.0 driver API
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="enter_pw")

In [13]:
def insert_data(query, rows, batch_size=10000):
    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(rows):
        res = conn.query(query, parameters = {'rows': rows[batch*batch_size:(batch+1)*batch_size].to_dict('records')}, db="neo4j")
        print(res)
        total += res[0]['total']
        batch += 1
        result = {"total": total, "batches": batch, "time": time.time()-start}
        print(result)
    return result

In [15]:
def insert_pois(rows, batch_size=10000):
    print(rows)
    query = '''
    UNWIND $rows AS row
    CREATE (p:PointOfInterest {name: row.names.local})
    CREATE (g:Geometry)
    SET g.location = point({latitude: toFloat(row.point[1]), longitude: toFloat(row.point[0]) })
    CREATE (g)<-[:HAS_GEOMETRY]-(p)
    SET g:Point
    CREATE (t:Tags)
    SET t += row.original_tags_dict
    CREATE (p)-[:HAS_TAGS]->(t)
    WITH *
    CALL apoc.create.addLabels(p, [row.class, row.subclass]) YIELD node
    RETURN COUNT(*) AS total
    '''
    return insert_data(query, rows, batch_size)

insert_pois(tempdf)

          geometry_id       class     subclass  \
0       n4840618776@4       local       suburb   
1       n8253475216@1  settlement       hamlet   
2        n150981480@4  settlement       hamlet   
3        n150965004@3  settlement       hamlet   
4       n8246746844@1  settlement       hamlet   
...               ...         ...          ...   
127745   w974480734@1  commercial    furniture   
127746   w622137659@1  commercial  convenience   
127747    w96330696@3     general     building   
127748    w97492542@5     amenity    fast_food   
127749    w31802851@2     amenity    fast_food   

                                                    names  \
0                                   {'local': 'La Jolla'}   
1                               {'local': 'Rancho Verde'}   
2                                  {'local': 'Hillsdale'}   
3                                       {'local': 'Agra'}   
4                              {'local': 'Sunrise Ranch'}   
...                              

{'total': 127750, 'batches': 13, 'time': 145.20623779296875}